In [1]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt

In [19]:
batchsize = 64
learning_rate = 0.001
epochs = 10

# Data Preparation

### preprocessing tranforms

In [3]:
transfrom = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ]
)

### loading dataset

In [5]:
class_names = ['plane', 'car','bird','cat','deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

In [10]:
train_dataset = torchvision.datasets.CIFAR10(root='./dataset',train=True,download=True,transform=transfrom)
test_dataset = torchvision.datasets.CIFAR10(root='./dataset',train=False,download=True,transform=transfrom)

train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batchsize, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=batchsize, shuffle=False)


# Design CNN Architecture

In [21]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.images_size = (32,32)
        self.w = 32
        self.h = 32
        self.input_channels = 3
        self.kernal_size = 3
        
        self.conv_layer1 = nn.Conv2d(self.input_channels, 32 , self.kernal_size); self.pool = nn.MaxPool2d(2,2)
        self.conv_layer2 = nn.Conv2d(32, 64 , self.kernal_size)
        self.conv_layer3 = nn.Conv2d(64, 64 , self.kernal_size) 

        self.fc1 = nn.Linear(64*4*4, 64)
        self.fc2 = nn.Linear(64, 10)
    

    
    def forward(self,x):
        x = self.pool(F.relu(self.conv_layer1(x)))
        x = self.pool(F.relu(self.conv_layer2(x)))
        x = (F.relu(self.conv_layer3(x)))
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


        

### create the model instance

In [22]:
cnn_model = CNN()

#### define the loss function and optimizer

In [23]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=learning_rate)


# Training Loop

In [27]:
n_steps = len(train_loader)
for epoch in range(epochs):
    running_loss = 0.0

    for i, (imgs,labels) in enumerate(train_loader):
        
        # Forward prop
        outputs = cnn_model(imgs)

        # Calc loss
        loss = loss_func(outputs,labels)

        # Backward prop

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f"[{epoch+1}] loss: f{running_loss/n_steps:.3f}")

print("Finished Training !")

path = './latest_cnn.pth'

torch.save(cnn_model.state_dict(),path)

[1] loss: f0.445
[2] loss: f0.423
[3] loss: f0.403
[4] loss: f0.380
[5] loss: f0.370
[6] loss: f0.351
[7] loss: f0.339
[8] loss: f0.319
[9] loss: f0.309
[10] loss: f0.296
Finished Training !


# Evaluating the model

### Load the saved model paramters

In [28]:
loaded_cnn = CNN()
loaded_cnn.load_state_dict(torch.load(path))
loaded_cnn.eval()

CNN(
  (conv_layer1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_layer2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv_layer3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1024, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)

### Calculate accuracy on test set

In [30]:
with torch.no_grad():
    n_correct = 0
    n_samples = len(test_loader.dataset)

    for imgs, labels in test_loader:

        outputs = loaded_cnn(imgs)

        _, preds = torch.max(outputs,1)
        
        n_correct += (preds == labels).sum().item()

    
    acc =(n_correct / n_samples) * 100.0
    print(f"Model Accuracy = {acc} %")

Model Accuracy = 71.54 %


# Test on examples out of the dataset 

In [33]:
new_transform = transforms.Compose(
    [
        transforms.Resize((32,32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ]
)

In [34]:
def load_img(img_path):
    img = Image.open(img_path)
    img = new_transform(img)
    img = img.unsqueeze(0)
    return img

In [35]:
images_paths  = ["airplane_ex1.jpeg","dog_ex2.jpeg","frog_ex3.jpeg"]
images = [load_img(img) for img in images_paths]


with torch.no_grad():
    for img in images:
        outputs = loaded_cnn(img)
        _, pred = torch.max(outputs,1)
        print(f"Prediction: {class_names[pred.item()]}")        

    
    

Prediction: plane
Prediction: dog
Prediction: frog
